In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.base import clone
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import euclidean_distances
from keras.preprocessing import image
from sklearn.cluster import DBSCAN
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Flatten, Input
import h5py
from sklearn.decomposition import IncrementalPCA
from time import time
from images_functions import print_photos_for_every_label
from helper_functions import get_batches
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.multioutput import ClassifierChain
from pca_functions import fitting_pca, pca_transform
import itertools
import config

In [ ]:
train_data = pd.read_pickle("reduced_result.csv")

#Delete NaNs
nans = pd.isnull(train_data).any(1).nonzero()[0]
train_data = train_data.drop(train_data.index[list(nans)])

#Getting labels in binary form
labels_temp = [l.split() for l in train_data["labels"]]
labels = np.array(list(map(lambda x:list(map(lambda y:int(y), x)), labels_temp)))
MB = MultiLabelBinarizer()
Y_train = MB.fit_transform(labels)

X_train = np.array([i for i in train_data["features"]])

In [ ]:
#Dirs with photos
TRAIN_DIR = config.TRAIN_DIR
#TEST_DIR = "D:/test_photos/"

#CSV files with photos ids and corresponding business ids 
#test_photo_to_biz = pd.read_csv("D:/test_photo_to_biz.csv")
train_photo_to_biz = config.train_photo_to_biz

#CSV file with labels corresponding to business
lbs_pd = config.labels

In [ ]:
batches, lbs = get_batches(train_photo_to_biz, lbs_pd)

In [ ]:
pca = IncrementalPCA(n_components=1712, batch_size = 1712)
f1 = h5py.File("data1.h5", "r")
datasets1 = sorted([i for i in f1], key=lambda x:int(x[5:]))
f2 = h5py.File("data2.h5", "r")
datasets2 = sorted([i for i in f2], key=lambda x:int(x[5:]))
f3 = h5py.File("data3.h5", "r")
datasets3 = sorted([i for i in f3], key=lambda x:int(x[5:]))
temp_data = 0
for i in tqdm(range(len(datasets1))):
    temp_data = fitting_pca(pca, f1, datasets1[i], temp_data, pca.n_components)
for i in tqdm(range(len(datasets2))):
    temp_data = fitting_pca(pca, f2, datasets2[i], temp_data, pca.n_components)
for i in tqdm(range(len(datasets3))):
    temp_data = fitting_pca(pca, f3, datasets3[i], temp_data, pca.n_components)
        
result_file = h5py.File("result_pca1712.h5", "w")

for i in tqdm(range(len(datasets1))):
    pca_transform(pca, f1, datasets1[i], 5, result_file)
for i in tqdm(range(len(datasets2))):
    pca_transform(pca, f2, datasets2[i], 5, result_file)
for i in tqdm(range(len(datasets3))):
    pca_transform(pca, f3, datasets3[i], 5, result_file)
    
f1.close()
f2.close()
f3.close()


result_file.close()



In [ ]:
file = h5py.File("model4_output.h5", "r")
businesses = file["businesses"][:]
imagenet_labels = file["labels"][:]
file.close()
lbs = imagenet_labels.reshape(234842, 1000)

In [ ]:
X = []
Y = []
n = 0
for i in np.unique(businesses):
    if type(list(lbs_pd[lbs_pd["business_id"]==i]["labels"])[0])!=float:
        Y.append(list(map(lambda x:int(x), list(lbs_pd[lbs_pd["business_id"]==i]["labels"])[0].split())))
        X.append(np.sum(lbs[np.where(businesses==i)], axis=0))
        n+=1
        print(n)
Y = MultiLabelBinarizer().fit_transform(Y)
X_imgnet = np.array(X)

In [ ]:
bs_ids = [i[0] for i in batches]
for i in range(len(businesses)):
    if businesses[i] not in bs_ids:
        lbs = np.delete(lbs,i,0)

In [ ]:
Path = "D:/dataset3/result_pca1712.h5"
file = h5py.File(Path, "r")
datasets = sorted([i for i in file], key=lambda x:int(x[5:]))
data = file[datasets[0]][:]
for i in range(1,len(datasets)):
    data = np.concatenate([data, file[datasets[i]][:]], axis=0)
file.close()

In [ ]:
f = h5py.File("kmeans_imgnet.h5","r")
kmeans_imgnet_labels = f["labels"][:]
f.close()

In [ ]:
f = h5py.File("kmeans1712.h5","r")
kmeans_labels = f["labels"][:]
f.close()

In [ ]:
def group_cluster(kmeans_labels, data, num):
    lbs_per_smpl = [[[] for j in range(8)] for i in range(len(batches))]
    n = 0
    for i in range(len(batches)):
        for j in range(len(batches[i][1])):
            lbs_per_smpl[i][kmeans_labels[n]].append(data[n])
            n+=1
    mean_per_lbl = [0 for i in range(len(batches))]
    for i in range(len(lbs_per_smpl)):
        if lbs_per_smpl[i]!=[]:
            mean_per_lbl[i] = list(map(lambda x:np.mean(x, axis=0) if x!=[] else 0, lbs_per_smpl[i]))
        else:
            mean_per_lbl[i] = []
        
    for i in range(len(mean_per_lbl)):
        n = 0
        for j in range(len(mean_per_lbl[i])):
            if type(mean_per_lbl[i][j-n])==int:
                del mean_per_lbl[i][j-n]
                n+=1
            
    X_cls = [0 for i in range(len(mean_per_lbl))]
    for i in range(len(mean_per_lbl)):
        X_cls[i] = np.concatenate(mean_per_lbl[i])
    
    for i in range(len(X_cls)):
        while(X_cls[i].shape[0]<num):
            X_cls[i] = np.concatenate([X_cls[i],X_cls[i]])
        X_cls[i] = X_cls[i][:num]
    
    X_cls = np.array(X_cls)
    return X_cls
X_cls = group_cluster(kmeans_labels, data, 13696)
X_imgnet_cls = group_cluster(kmeans_imgnet_labels, lbs, 8000)

In [ ]:
mlp_classifier = MLPClassifier(solver='lbfgs', max_iter=300, 
                               hidden_layer_sizes=(1024,512,128,32,16,))

xgb_classifier = OneVsRestClassifier(XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"))

svm_classifier = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))

mlp_classifier_clst = MLPClassifier(solver='lbfgs', max_iter=300, hidden_layer_sizes=(1024,512,128,32,16,))

xgb_classifier_clst = OneVsRestClassifier(XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"))

svm_classifier_clst = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))

mlp_ensemble = MLPClassifier(solver='sgd', max_iter=5, hidden_layer_sizes=(74,44,))

cls_chain1 = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[0,6,2,1,5,7,4,8,3])

cls_chain2 = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[1,2,6,5,8,4,7,0,3])

cls_chain3 = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[7,2,6,5,3,4,1,8,0])

cls_chain4 = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[4,2,1,5,3,7,8,6,0])

cls_chain5 = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[5,2,7,1,6,8,4,0,3])

cls_chain1_clst = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[0,6,2,1,5,7,4,8,3])

cls_chain2_clst = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[1,2,6,5,8,4,7,0,3])

cls_chain3_clst = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[7,2,6,5,3,4,1,8,0])

cls_chain4_clst = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[4,2,1,5,3,7,8,6,0])

cls_chain5_clst = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[5,2,7,1,6,8,4,0,3])

mlp_classifier_img = MLPClassifier(solver='lbfgs', max_iter=300, hidden_layer_sizes=(1024,512,128,32,16,))

xgb_classifier_img = OneVsRestClassifier(XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"))

svm_classifier_img = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))

mlp_classifier_clst_img = MLPClassifier(solver='lbfgs', max_iter=300, hidden_layer_sizes=(1024,512,128,32,16,))

xgb_classifier_clst_img = OneVsRestClassifier(XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"))

svm_classifier_clst_img = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))

mlp_ensemble_img = MLPClassifier(solver='sgd', max_iter=5, hidden_layer_sizes=(74,44,))

cls_chain1_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[0,6,2,1,5,7,4,8,3])

cls_chain2_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[1,2,6,5,8,4,7,0,3])

cls_chain3_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[7,2,6,5,3,4,1,8,0])

cls_chain4_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[4,2,1,5,3,7,8,6,0])

cls_chain5_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[5,2,7,1,6,8,4,0,3])

cls_chain1_clst_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[0,6,2,1,5,7,4,8,3])

cls_chain2_clst_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[1,2,6,5,8,4,7,0,3])

cls_chain3_clst_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[7,2,6,5,3,4,1,8,0])

cls_chain4_clst_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[4,2,1,5,3,7,8,6,0])

cls_chain5_clst_img = ClassifierChain(base_estimator=XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6,
                                                   n_estimators=100, objective="multi:softmax"), order=[5,2,7,1,6,8,4,0,3])

In [ ]:
def make_predictions(classifier, X_train, Y_train):
    
    results = []
    label_results = [] #Results for each label separately
    predictions = []
    
    beg = 1900
    end = 1996
    
    #Testing on each part of data set with re-training
    for i in tqdm(range(1)):
        
        x_train = np.concatenate([X_train[:beg],X_train[end:]], axis=0)
        y_train = np.concatenate([Y_train[:beg],Y_train[end:]], axis=0)
        
        x_test = X_train[beg:end]
        y_test = Y_train[beg:end]
    
        classifier.fit(x_train, y_train)
    
        y_pred = classifier.predict(x_test)
        
        predictions.append(y_pred)
        
        results.append(f1_score(y_test, y_pred, average="micro"))
    
        label_results.append([f1_score(list(map(lambda x:x[i], y_test)),list(map(lambda x:x[i], y_pred)), average="micro") for i in range(9)])
    
        end = beg
        beg-=100
    
        classifier = clone(classifier)
    return results, label_results, predictions

In [ ]:
classifiers = [mlp_classifier, mlp_classifier_clst, xgb_classifier, xgb_classifier_clst, svm_classifier, svm_classifier_clst,
               cls_chain1, cls_chain1_clst, cls_chain2, cls_chain2_clst, cls_chain3, cls_chain3_clst, cls_chain4, 
               cls_chain4_clst, cls_chain5, cls_chain5_clst,
              mlp_classifier_img, mlp_classifier_clst_img, xgb_classifier_img, xgb_classifier_clst_img, svm_classifier_img, svm_classifier_clst_img,
               cls_chain1_img, cls_chain1_clst_img, cls_chain2_img, cls_chain2_clst_img, cls_chain3_img, cls_chain3_clst_img, cls_chain4_img, 
               cls_chain4_clst_img, cls_chain5_img, cls_chain5_clst_img]


classifiers_str = ["mlp_classifier", "mlp_classifier_clst", "xgb_classifier", "xgb_classifier_clst", "svm_classifier", 
                   "svm_classifier_clst", "cls_chain1", "cls_chain1_clst", "cls_chain2", "cls_chain2_clst", "cls_chain3", 
                   "cls_chain3_clst", "cls_chain4", "cls_chain4_clst", "cls_chain5", "cls_chain5_clst",
                  "mlp_classifier_img", "mlp_classifier_clst_img", "xgb_classifier_img", "xgb_classifier_clst_img", "svm_classifier_img", 
                   "svm_classifier_clst_img", "cls_chain1_img", "cls_chain1_clst_img", "cls_chain2_img", "cls_chain2_clst_img", "cls_chain3_img", 
                   "cls_chain3_clst_img", "cls_chain4_img", "cls_chain4_clst_img", "cls_chain5_img", "cls_chain5_clst_img"]


f = h5py.File("classifiers_result.h5", "w")


labels = ["good_for_lunch","good_for_dinner","takes_reservations","outdoor_seating","restaurant_is_expensive","has_alcohol",
          "has_table_service","ambience_is_classy","good_for_kids"]

In [ ]:
#Results
def print_results(label_results):
    
    print("")
    
    mean_label_result = np.mean(label_results, axis=0)

    for i in range(9):
        
        print(labels[i]+" : "+str(mean_label_result[i])+"\n")

    print("Total: "+str(np.mean(results)))

In [ ]:
for i in range(16,len(classifiers)):
    
    if "clst_img" in classifiers_str[i]:
        
        results, label_results, predictions = make_predictions(classifiers[i], X_imgnet_cls, Y)
    
    elif "clst" in classifiers_str[i]:
        
        results, label_results, predictions = make_predictions(classifiers[i], X_cls, Y_train)
    
    elif "img" in classifiers_str[i]:
        
        results, label_results, predictions = make_predictions(classifiers[i], X_imgnet, Y)
    
    else:
        
        results, label_results, predictions = make_predictions(classifiers[i], X_train, Y_train)
    
    preds = np.concatenate(predictions, axis=0)
    
    lbs_res = np.concatenate(label_results, axis=0)
    
    print(f"\n{classifiers_str[i]}\n")
    
    print_results(label_results)
    
    f.create_dataset(f"{classifiers_str[i]}_preds", data=preds)
    
    f.create_dataset(f"{classifiers_str[i]}_lbs", data=lbs_res)

f.close()

In [ ]:
f.close()

In [ ]:
print_photos_for_every_label(labels, predictions[0], Y_train[1900:], batches, TRAIN_DIR)

In [ ]:
print("Correlation between labels")

corr = list(map(lambda x:list(map(lambda y:str(np.round(y,2)) ,x)) ,list(np.corrcoef(Y_train.T))))

for i in range(9):
    
    print(labels[i], i)

print("    "+"        ".join([str(i) for i in range(9)]))

for i in range(9):
    
    print(i, list(map(lambda x:x+("00" if len(x)<4 else "0" if len(x)<5 else ""),corr[i])))

In [ ]:
labels = ["good_for_lunch","good_for_dinner","takes_reservations","outdoor_seating","restaurant_is_expensive","has_alcohol","has_table_service","ambience_is_classy","good_for_kids"]

print("Percentage of each lable:\n")

for i in range(9):
    
    print(labels[i]+" "*(25-len(labels[i])), np.round(np.count_nonzero(Y_train.T[i])/1996, 3))
    print("\n")

In [ ]:
print("Percentage of each pair of labels")

for i in range(9):
    
    print("\n"+labels[i], i)

pair_matrix = [[0 for i in range(10)] for j in range(10)]

for i in range(1996):
    
    indxs = np.where(Y_train[i]==1)[0]
    
    pairs = list(itertools.combinations(sorted(indxs), 2))
    
    for i in pairs:
        
        pair_matrix[i[0]][i[1]]+=1

print("\n\n\n")

for i in range(10):
    
    for j in range(10):
        
        if pair_matrix[i][j]!=0:
            
            print(f"\n{i} and {j} : ", np.round(pair_matrix[i][j]/1996, 3))